## El objetivo principal es lograr construir un modelo que sea sumamente efectio en la recomendación de peliculas para un usuario en expecifico.

In [2]:
# Importación de Librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
# Cargamos los datos
df_movies = pd.read_csv(r'data/movies.csv')
df_rating = pd.read_csv(r'data/ratings.csv')

In [4]:
df_movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
df_rating.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [6]:
# Verificamos los duplicados
print(f"El Dataframe que contiene los registros de las peliculas, cuenta con {df_movies.duplicated().sum()} duplicados.")
print(f"El Dataframe que contiene los registros de las raking, cuenta con {df_movies.duplicated().sum()} duplicados.")

El Dataframe que contiene los registros de las peliculas, cuenta con 0 duplicados.
El Dataframe que contiene los registros de las raking, cuenta con 0 duplicados.


In [7]:
# Verificamos los NaN
df_movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
# Verificamos los NaN
df_rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [9]:
# Unimos los dos DataFrames
data = df_rating.merge(df_movies, on='movieId')

### En este enfoque, las recomendaciones se hacen en función de las preferencias de otros usuarios que son similares al usuario objetivo. En tu caso, estás buscando usuarios que son similares (basándote en la similitud del coseno de sus calificaciones de películas) y luego recomendando películas que esos usuarios similares han calificado pero que el usuario objetivo no ha visto.

In [12]:
# Dividimos los datos en un conjunto de entrenamiento y un conjunto de prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convertimos el DataFrame en una matriz de usuario-película con las calificaciones como valores
ratings_matrix = train_data.pivot_table(index='userId', columns='title', values='rating')

# Rellenamos los valores NaN con 0, ya que la ausencia de calificación se puede interpretar como una calificación de 0
ratings_matrix_filled = ratings_matrix.fillna(0)

# Calculamos la similitud de coseno entre los usuarios
user_similarity = cosine_similarity(ratings_matrix_filled)

# Creamos un DataFrame de similitudes
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix_filled.index, columns=ratings_matrix_filled.index)

# Utilizamos k-NN para encontrar los vecinos más cercanos basados en la similitud de coseno
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
knn.fit(user_similarity_df)

# Definimos una función para recomendar películas basadas en la similitud de usuarios
def recomendar_peliculas_colaborativas(user_id):
    # Obtenemos los usuarios similares al usuario dado
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:11].index
    
    # Obtenemos las películas que los usuarios similares han calificado
    peliculas_calificadas = train_data[train_data['userId'].isin(similar_users)]['title'].tolist()
    
    # Obtenemos las películas que el usuario dado no ha calificado
    peliculas_no_calificadas = set(peliculas_calificadas) - set(train_data[train_data['userId'] == user_id]['title'].tolist())
    
    # Devolvemos las películas no calificadas como recomendaciones
    return list(peliculas_no_calificadas)

def calculate_recall(user_id, recommendations, test_data):
    # Obtenemos las películas que el usuario ha calificado en el conjunto de prueba
    test_movies = test_data[test_data['userId'] == user_id]['title'].tolist()
    
    # Calculamos el recall comparando las recomendaciones con las películas calificadas en el conjunto de prueba
    recall = recall_score([movie in test_movies for movie in recommendations], [True]*len(recommendations))
    
    return recall

def calculate_f1_score(user_id, recommendations, test_data):
    # Obtenemos las películas que el usuario ha calificado en el conjunto de prueba
    test_movies = test_data[test_data['userId'] == user_id]['title'].tolist()
    
    # Calculamos el F1 score comparando las recomendaciones con las películas calificadas en el conjunto de prueba
    f1 = f1_score([movie in test_movies for movie in recommendations], [True]*len(recommendations))
    
    return f1

# Ejemplo de uso:
user_id = int(input("Ingresa el UserId para las recomendaciones: "))
peliculas_recomendadas = recomendar_peliculas_colaborativas(user_id)
print(f"Top 5 películas recomendadas para UserId {user_id}: {peliculas_recomendadas[:5]}")

# Ahora, podemos verificar el rendimiento de nuestro sistema de recomendación
# Primero, obtenemos todas las películas que el usuario ha calificado en el conjunto de prueba
peliculas_reales = test_data[test_data['userId'] == user_id]['title'].tolist()

# Luego, calculamos la precisión como la proporción de recomendaciones que están en la lista de películas reales
precision = len(set(peliculas_recomendadas) & set(peliculas_reales)) / len(peliculas_recomendadas)
print(f"La precisión del sistema de recomendación para el UserId {user_id} es: {precision}")
f1 = calculate_f1_score(user_id, peliculas_recomendadas, test_data)
print(f"El F1 score del algoritmo de recomendación para el UserId {user_id} es: {f1}")

Top 5 películas recomendadas para UserId 602: ['Jack (1996)', 'American President, The (1995)', 'Toy Story (1995)', 'Richard III (1995)', 'Winnie the Pooh and the Blustery Day (1968)']
La precisión del sistema de recomendación para el UserId 602 es: 0.20535714285714285
El F1 score del algoritmo de recomendación para el UserId 602 es: 0.34074074074074073


### En este enfoque, hago recomendaciones en función de la similitud entre los ítems (en este caso, las películas). Para un usuario dado, encuentro películas que son similares a las que el usuario ha calificado altamente en el pasado y recomiendo esas películas.

In [ ]:
# Crear una tabla pivot con usuarios como filas y películas como columnas
pivot_table = data.pivot_table(index='userId', columns='title', values='rating')

# Rellenar los valores NA con 0
pivot_table.fillna(0, inplace=True)

# Calcular la similitud del coseno entre las películas
item_cosine_sim = cosine_similarity(pivot_table.T)

# Convertir los resultados a un DataFrame
item_cosine_sim_df = pd.DataFrame(item_cosine_sim, index=pivot_table.columns, columns=pivot_table.columns)

def recommend_movies(user_id, num_recommendations=5):
    # Obtener las películas que el usuario ha calificado
    user_movies = data[data['userId'] == user_id]['title']
    
    # Calcular la similitud total de todas las películas que el usuario ha calificado
    total_similarity = item_cosine_sim_df[user_movies].sum(axis=1)
    
    # Calcular la puntuación de recomendación para cada película
    recommendation_scores = total_similarity / len(user_movies)
    
    # Ordenar las películas por puntuación de recomendación y devolver las más altas
    recommended_movies = recommendation_scores.sort_values(ascending=False)
    
    return recommended_movies.index[:num_recommendations]

# Probar la función con un ID de usuario
print(recommend_movies(32))

Index(['Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'Batman (1989)',
       'Mission: Impossible (1996)', 'Independence Day (a.k.a. ID4) (1996)',
       'Toy Story (1995)'],
      dtype='object', name='title')


## Reflexiones
El modelo actual presenta un amplio margen para mejoras. Para lograrlo, necesito enfocarme en la optimización del modelo y en la mejora de la base de datos con la que estoy trabajando.
A pesar de que el porcentaje de precisión puede parecer bajo, es importante destacar que el algoritmo demuestra su capacidad para ofrecer recomendaciones de películas que el usuario no ha visto anteriormente. Esto indica que el sistema está funcionando correctamente en su objetivo principal de descubrir y sugerir contenido nuevo para el usuario.